<a href="https://colab.research.google.com/github/johannakin/DD2424-Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hej emma och alice")

hej emma och alice
